<a href="https://colab.research.google.com/github/Yospy/Tweet-Sentiment-Extraction-using-BERT/blob/main/Tweet_Sentiment_Extraction_using_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow-text
import pandas as pd
from numpy import *
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.6 MB 5.0 MB/s 
     |████████████████████████████████| 511.7 MB 6.9 kB/s 
     |████████████████████████████████| 438 kB 68.6 MB/s 
     |████████████████████████████████| 5.8 MB 59.4 MB/s 
     |████████████████████████████████| 1.6 MB 58.9 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninst

In [3]:
train = pd.read_csv('/content/train.csv')
train.head()

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [4]:
test = pd.read_csv('/content/test.csv')
test.head()

,textID,text,sentiment
0,f87dea47db,Last session of the day http://twitpic.com/67ezh,neutral
1,96d74cb729,Shanghai is also really exciting (precisely -...,positive
2,eee518ae67,"Recession hit Veronique Branquinho, she has to...",negative
3,01082688c6,happy bday!,positive
4,33987a8ee5,http://twitpic.com/4w75p - I like it!!,positive


In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16436 entries, 0 to 16435
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   textID         16436 non-null  object
 1   text           16435 non-null  object
 2   selected_text  16434 non-null  object
 3   sentiment      16435 non-null  object
dtypes: object(4)
memory usage: 513.8+ KB


In [6]:
train.describe()

,textID,text,selected_text,sentiment
count,16436,16435,16434,16435
unique,16436,16435,13700,3
top,cb774db0d1,"I`d have responded, if I were going",good,neutral
freq,1,1,125,6671


In [7]:
train.isnull().sum()

textID           0
text             1
selected_text    2
sentiment        1
dtype: int64

In [8]:
train = train.dropna()

In [9]:
train.head()

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [10]:
train.sentiment.unique()

array(['neutral', 'negative', 'positive'], dtype=object)

In [11]:
train = train.drop(['selected_text'],1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


In [12]:
train.head()

,textID,text,sentiment
0,cb774db0d1,"I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,negative
2,088c60f138,my boss is bullying me...,negative
3,9642c003ef,what interview! leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...",negative


In [13]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

In [14]:
train.sentiment = encoder.fit_transform(train.sentiment)

In [15]:
train.head()

,textID,text,sentiment
0,cb774db0d1,"I`d have responded, if I were going",1
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,0
2,088c60f138,my boss is bullying me...,0
3,9642c003ef,what interview! leave me alone,0
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...",0


In [16]:
encoder.classes_

array(['negative', 'neutral', 'positive'], dtype=object)

In [17]:
train.sentiment.unique()

array([1, 0, 2])

In [18]:
train.groupby('sentiment').describe()

textID                          text         \
           count unique         top freq count unique   
sentiment                                               
0           4608   4608  549e992a42    1  4608   4608   
1           6670   6670  cb774db0d1    1  6670   6670   
2           5156   5156  6e0c6d75b1    1  5156   5156   

                                                                   
                                                         top freq  
sentiment                                                          
0              Sooo SAD I will miss you here in San Diego!!!    1  
1                        I`d have responded, if I were going    1  
2          2am feedings for the baby are fun when he is a...    1

In [19]:
df_neutral = train[train['sentiment']==0]
df_neutral.shape

(4608, 3)

In [20]:
df_negative = train[train['sentiment']==1]
df_negative.shape

(6670, 3)

In [21]:
df_positive = train[train['sentiment']==2]
df_positive.shape

(5156, 3)

In [22]:
df_neg_bal = df_negative.sample(df_neutral.shape[0])
df_neg_bal.shape

(4608, 3)

In [23]:
df_posi_bal = df_positive.sample(df_neutral.shape[0])
df_posi_bal.shape

(4608, 3)

In [24]:
train_bal = pd.concat([df_neg_bal,df_posi_bal,df_neutral])
train_bal

,textID,text,sentiment
9259,7401c4869e,Going to take it quite easy today actually. Ye...,1
6933,b3beff7cdd,_autism BUT on the bright side you`re on twitt...,1
14550,9ba59ca56d,same with drunk guests.. Our jobs are pretty ...,1
617,a82816a060,no one I know likes boiled peanuts t.,1
4845,148d78d525,Going to revise for physics.,1
...,...,...,...
16418,a43ba3e5b2,Stood outsiden got the WORST butterflies everr...,0
16426,cb45fae109,listening to AM- teddy picker She saw it and ...,0
16429,ca4c090ff4,33 minutes left before my flannel hobbit shoes...,0
16433,4a69be0209,ONE OF THE WORST DAYS OF MY LIFE.,0


In [25]:
train_bal.sentiment.unique()

array([1, 2, 0])

In [26]:
train_bal.groupby('sentiment').describe()

textID                          text         \
           count unique         top freq count unique   
sentiment                                               
0           4608   4608  549e992a42    1  4608   4608   
1           4608   4608  7401c4869e    1  4608   4608   
2           4608   4608  59f3cb342d    1  4608   4608   

                                                                   
                                                         top freq  
sentiment                                                          
0              Sooo SAD I will miss you here in San Diego!!!    1  
1          Going to take it quite easy today actually. Ye...    1  
2           http://tinyurl.com/d7tb38 Nice Photoshop Effects    1

In [27]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_bal.text,train_bal.sentiment,test_size=0.2)

In [28]:
X_train

2045      I am so sorry I missed your tweet  I was on a...
14416               Ok super bored... guess no one else it
11022    Cleaned four bathrooms this afternoon....(yes,...
7438      Yes I did see Mitch Hedberg and the whole flo...
12152    The stratus is going bye bye today  i loved th...
                               ...                        
14453     My friend and I have a request that you play ...
8297     Just sneezed three times in quick succession. ...
13869                           Hi there! Nice to meet you
6092           'now or never' makes me cry everytime  lol.
14055     happy birthday month!! I only get a day but y...
Name: text, Length: 11059, dtype: object

In [29]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_model = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [30]:
bert_model(bert_preprocess(['Yash is a mad ass',"ahh asfuck shit"]))['pooled_output']

<tf.Tensor: shape=(2, 768), dtype=float32, numpy=
array([[-0.69555235, -0.2541849 ,  0.6425888 , ...,  0.49627337,
        -0.4975744 ,  0.7456868 ],
       [-0.8527228 , -0.2978963 , -0.29581654, ...,  0.00763623,
        -0.61723346,  0.85679114]], dtype=float32)>

In [31]:
inputs = tf.keras.layers.Input(shape=(), dtype=tf.string,name='text')
preprocess_t = bert_preprocess(inputs)
output = bert_model(preprocess_t)

In [32]:
l = tf.keras.layers.Dropout(0.3)(output['pooled_output'])
l = tf.keras.layers.Dense(3, activation='sigmoid', name='output')(l)

In [33]:
model = tf.keras.Model(inputs=[inputs],outputs=[l])

In [34]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_type_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128),                                                          
                                 'input_word_ids':                                                
                                (None, 128)}                                                  

In [35]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['acc'])

In [37]:
model.fit(X_train,y_train,epochs=15)

Epoch 1/15
346/346 [==============================] - 106s 306ms/step - loss: 1.1427 - acc: 0.3812
Epoch 2/15
346/346 [==============================] - 111s 321ms/step - loss: 1.0540 - acc: 0.4548
Epoch 3/15
346/346 [==============================] - 111s 320ms/step - loss: 1.0202 - acc: 0.4875
Epoch 4/15
346/346 [==============================] - 110s 319ms/step - loss: 0.9900 - acc: 0.5066
Epoch 5/15
346/346 [==============================] - 110s 318ms/step - loss: 0.9770 - acc: 0.5175
Epoch 6/15
346/346 [==============================] - 109s 316ms/step - loss: 0.9685 - acc: 0.5295
Epoch 7/15
346/346 [==============================] - 110s 317ms/step - loss: 0.9628 - acc: 0.5305
Epoch 8/15
346/346 [==============================] - 110s 317ms/step - loss: 0.9628 - acc: 0.5290
Epoch 9/15
346/346 [==============================] - 109s 316ms/step - loss: 0.9578 - acc: 0.5305
Epoch 10/15
346/346 [==============================] - 110s 317ms/step - loss: 0.9577 - acc: 0.5379
Epoch 11/

In [38]:
test_data = test.drop(['sentiment','textID'],1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


In [39]:
model.predict(X_test)

87/87 [==============================] - 29s 324ms/step


array([[0.64088666, 0.7789036 , 0.43067732],
       [0.77020055, 0.56184417, 0.5654379 ],
       [0.6292827 , 0.48049882, 0.7169028 ],
       ...,
       [0.5295032 , 0.5533195 , 0.466734  ],
       [0.73889226, 0.57372785, 0.60989547],
       [0.49906808, 0.54700106, 0.7614707 ]], dtype=float32)

In [40]:
model.evaluate(X_test,y_test)

87/87 [==============================] - 28s 317ms/step - loss: 0.8841 - acc: 0.5946


[0.8841243982315063, 0.594575047492981]

In [41]:
y_pred = model.predict(test_data)

111/111 [==============================] - 35s 318ms/step


In [42]:
y_pred = argmax(y_pred,axis=1)

In [43]:
y_pred = pd.DataFrame(y_pred)

In [44]:
y_pred

,0
0,2
1,2
2,0
3,2
4,2
...,...
3529,0
3530,2
3531,2
3532,2
